In [ ]:
%matplotlib inline
from datetime import date, datetime, timedelta, time
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy

In [ ]:
import json
import numpy
from numpy import log, exp, pi
import scipy.stats, scipy
import pymultinest
import matplotlib.pyplot as plt

# we define the problem: we need a prior function which maps from [0:1] to the parameter space

# we only have one parameter, the position of the gaussian (ndim == 1)
# map it from the unity interval 0:1 to our problem space 0:2 under a uniform prior
def prior(cube, ndim, nparams):
	cube[0] = cube[0] * 2

# our likelihood function consists of 6 gaussians modes (solutions) at the positions
gauss_means = numpy.array([0.1, 0.2, 0.5, 0.55, 0.9, 1.1])

width = 0.01

def loglike(cube, ndim, nparams):
	# get the current parameter (is between 0:2 now)
	pos = cube[0]
	likelihood = exp(-0.5 * ((pos - gauss_means ) / width)**2) / (2*pi*width**2)**0.5
	return log(likelihood.mean())

# number of dimensions our problem has
parameters = ["position"]
n_params = len(parameters)

# run MultiNest
pymultinest.run(loglike, prior, n_params, outputfiles_basename='out/test',
                resume = False, verbose = True,
               sampling_efficiency=0.3,
               importance_nested_sampling = False,
               mode_tolerance=-4
               )
json.dump(parameters, open('out/params.json', 'w')) # save parameter names

# now run the script and analyse the output using multinest_marginals.py::
# 
#    $ python 1_1d_multimodal.py &&  multinest_marginals.py 1_1d_multimodal_out
#
# then open the file 1_1d_multimodal_outmarg.pdf
#
# Btw, ln(ev) should be ln(1 / 2)

In [ ]:
ngrid = 5000
x = np.linspace(-.5,2,ngrid)

In [ ]:
y = np.array([loglike([xpt,],1,1) for xpt in x])

In [ ]:
a = pymultinest.Analyzer(n_params=n_params, outputfiles_basename='out/test')

In [ ]:
a.get_data().shape

In [ ]:
a.data

In [ ]:
np.log(0.5)

In [ ]:
a.get_stats()

In [ ]:
a.get_best_fit()

In [ ]:
modes = a.get_mode_stats()['modes']
len(modes)

In [ ]:
modes

In [ ]:
plotter = pymultinest.PlotMarginalModes(a)

In [ ]:
# plt.plot(x,y)
plt.plot(x,np.exp(y))
# plt.yscale('log')
# xlim=plt.xlim(0.4,0.6)
# plt.ylim(-100,0)

In [ ]:
plotter.plot_marginal(0, with_ellipses=True, with_points=True, grid_points=500)
for m in sorted(modes, key=lambda m:m['mean']):
    m_mean = m['mean']
    plt.axvline(m_mean, ls=':')
    print m_mean
# plt.yscale('log')
# plt.xlim(*xlim)